In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import dit_fast_attention
import os
file_path=os.path.abspath(dit_fast_attention.__file__)
dir_path=os.path.dirname(file_path)
os.chdir(dir_path)
from diffusers import DiTPipeline, DPMSolverMultistepScheduler
import argparse
import torch
from evaluation import test_latencies
from dit_fast_attention import transform_model_fast_attention
import numpy as np
import os
import time
import thop
from utils import calculate_flops
import re
import seaborn as sns
# sns.set_style("whitegrid")

from collections import namedtuple
os.environ["https_proxy"]="http://10.10.20.100:1089"


In [ ]:
from torch import tensor
def parse_txt_data(txt_data):
    data = []
    for block in txt_data.split('\n\n'):
        lines = block.strip().split('\n')
        for line in lines:
            if line.startswith('Namespace'):
                namespace = re.findall(r"Namespace\((.*)\)", line)[0]
                namespace_args = [arg.split('=') for arg in namespace.split(', ')]
                namespace_dict = {arg[0]: eval(arg[1]) for arg in namespace_args}
                # namespace_obj = namedtuple('Namespace', namespace_dict.keys())(**namespace_dict)
                data.append(namespace_dict)
            # elif line.startswith('calib_ssim'):
            #     calib_ssim = float(line.split('=')[1])
            #     # data.append(calib_ssim)
            #     namespace_dict["calib_ssim"] = calib_ssim
            elif line.startswith('{'):
                metrics = eval(line)
                # data.append(metrics)
                namespace_dict["metrics"] = metrics
            elif line.startswith('macs'):
                macs = eval(line.split('=')[1])
                namespace_dict["macs"] = macs
                # data.append(macs)
            elif line.startswith('attn_mac'):
                attn_mac = float(line.split('=')[1])
                # data.append(attn_mac)
                namespace_dict["attn_mac"] = attn_mac
            elif line.startswith('latencies'):
                latencies = eval(line.split('=')[1])
                # data.append(latencies)
                namespace_dict["latencies"] = latencies
                # print(latencies)
    return data

txt_data=open("output/results.txt").read()
data=parse_txt_data(txt_data)


In [ ]:
# Ablation plot
seqs=[1,0.975,0.95,0.925,0.9,0.875,0.85]
seqs=[0.975,0.95,0.925,0.9,0.875,0.85]
# sfig, saxes = plt.subplots(1,3,figsize=(3.8*3,1.5))
fig=plt.figure(figsize=(6,2))
ax=fig.add_subplot(121)
# titles=["512x512 DiT-XL","1024x1024 PixArt-Sigma-XL","2048x2048 PixArt-Sigma-XL"]

raw_macs=2705/1e3
raw_fid=30.868
raw_is=210.0112
for modeli, model_name in enumerate(["facebook/DiT-XL-2-512"]):
    nbatch=8 if model_name=="facebook/DiT-XL-2-512" else 1
    
    for ablation in ["","full_attn+cfg_attn_share","residual_window_attn","output_share"]:
        # lats_all=[]
        # lats_attn=[]
        if ablation=="output_share":
            seqs=seqs[1:]
        iss=[raw_is]
        attn_macs=[raw_macs]
        visited=[]
        for threshold in seqs:
            for l in data[::-1]:
                # print(l)
                if l["model"]==model_name and l["n_calib"]==8 and l["n_steps"]==20 and \
                    l["eval_n_images"]==5000 and "attn_mac" in l and l["threshold"]==threshold \
                        and l.get("ablation","")==ablation:
                    # if "metrics" not in l or not "IS" in l["metrics"]:
                    #     continue
                    if "metrics" not in l or not "IS" in l["metrics"]:
                        continue
                    if l["threshold"] in visited:
                        continue
                    visited.append(l["threshold"])
                    attn_macs.append(l["attn_mac"]/1e3)
                    iss.append(l["metrics"]["IS"][0].item())
                    # lats_all.append(l["latencies"][f"{nbatch}_all"])
                    # lats_attn.append(l["latencies"][f"{nbatch}_attn"])
        ablation="DiTFastAttn" if ablation=="" else ablation
        ablation=ablation.replace("full_attn+","").replace("cfg_attn_share","ASC").replace("residual_window_attn","WA").replace("output_share","AST")    
        ax.plot(attn_macs, iss, label=f"{ablation}")
        # ax.plot(attn_macs, lats_attn, label="Attention")
        # ax.set_title(titles[modeli])
        ax.legend()
        ax.set_ylabel("Inception Score")
        ax.set_xlabel("Attention TFLOPs")
        # print(f"{model_name} IS",iss)
        # print(f"{model_name} Attention",attn_macs)
        print(iss)
        print(attn_macs)

plt.ylim(190,211)
plt.xlim(1.75,2.75)
# sfig.show()
# fig.savefig("plot/ablation.pdf", bbox_inches='tight')
    

In [ ]:
pipe = DiTPipeline.from_pretrained("facebook/DiT-XL-2-512", torch_dtype=torch.float16).to("cuda")
calib_x=torch.randint(0, 1000, (1,),generator=torch.Generator().manual_seed(3)).to("cuda")
n_stepss=[20,30,40,50,60]

steps_macs=[]
for n_steps in n_stepss:
    _,a_macs=calculate_flops(pipe,calib_x,n_steps=n_steps)
    steps_macs.append(a_macs/1e3)

In [ ]:
# Ablation plot
seqs=[1,0.975,0.95,0.925,0.9,0.875,0.85]
seqs=[0.975,0.95,0.925,0.9,0.875,0.85]
# sfig, saxes = plt.subplots(1,3,figsize=(3.8*3,1.5))
n_stepss=[20,30,40]
ax=fig.add_subplot(122)
# titles=["512x512 DiT-XL","1024x1024 PixArt-Sigma-XL","2048x2048 PixArt-Sigma-XL"]

for modeli, model_name in enumerate(["facebook/DiT-XL-2-512"]):
    nbatch=8 if model_name=="facebook/DiT-XL-2-512" else 1
    for n_steps in n_stepss:
        iss=[]
        attn_macs=[]
        visited=[]
        for threshold in seqs:
            for l in data[::-1]:
                # print(l)
                if l["model"]==model_name and l["n_calib"]==8 and l["n_steps"]==n_steps and \
                    l["eval_n_images"]==5000 and "attn_mac" in l and l["threshold"]==threshold and l.get("ablation","")=="":
                    # if "metrics" not in l or not "IS" in l["metrics"]:
                    #     continue
                    if "metrics" not in l or not "IS" in l["metrics"]:
                        continue
                    if l["threshold"] in visited:
                        continue
                    visited.append(l["threshold"])
                    attn_macs.append(l["attn_mac"]/1e3)
                    iss.append(l["metrics"]["IS"][0].item())
                    # lats_all.append(l["latencies"][f"{nbatch}_all"])
                    # lats_attn.append(l["latencies"][f"{nbatch}_attn"])
        raw_macs=steps_macs[n_stepss.index(n_steps)]
        if len(iss)==0:
            continue
        raw_is=iss[0]
        iss=[raw_is]+iss
        attn_macs=[raw_macs]+attn_macs
        attn_macs=np.array(attn_macs)/raw_macs
        ax.plot(attn_macs, iss, label=f"steps={n_steps}")
        # ax.plot(attn_macs, lats_attn, label="Attention")
        # ax.set_title(titles[modeli])
        ax.legend()
        ax.set_ylabel("Inception Score")
        ax.set_xlabel("Attention TFLOPs Fraction")
        # print(f"{model_name} IS",iss)
        # print(f"{model_name} Attention",attn_macs)
        print(iss)
        print(attn_macs)
fig.tight_layout()
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=2, hspace=2)

# plt.ylim(190,211)
# plt.xlim(1.75,2.75)
# sfig.show()
fig.savefig("plot/ablation.jpg", bbox_inches='tight')
fig.savefig("plot/ablation.pdf", bbox_inches='tight')
    